In [1]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.

# Python native
import pickle
from statistics import median as median
from tqdm import tqdm
import os
from platform import system
if system().lower() == "windows":
    os.chdir("c:\\Users\\Tim\\Development\\OCELFeatureExtractionExperiments")
elif system().lower() == "linux":
    os.chdir("/home/tim/Development/OCELFeatureExtractionExperiments/")
from copy import deepcopy

# Data handling
import pandas as pd
import numpy as np

# Object centric process mining
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# PyG
import torch
from ocpa_PyG_integration.EventGraphDataset import EventGraphDataset
from torch_geometric.loader import DataLoader

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# Global variables
from experiment_config import STORAGE_PATH, RANDOM_SEED, TARGET_LABEL

# Visualization
import matplotlib.pyplot as plt

# Custom models
from model import GCN, GAT

# Model evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.13.1+cpu
Cuda available: False
Torch geometric version: 2.2.0


In [2]:
ds_train = EventGraphDataset(
    root="data/ocpa-processed/",
    filename="BPI2017-feature_storage-split.pkl",
    label_key=("event_remaining_time", ()),
    train=True,
)

Processing...
22056it [01:21, 269.77it/s]
Done!


In [3]:
ds_test = EventGraphDataset(
    root="data/ocpa-processed/",
    filename="BPI2017-feature_storage-split.pkl",
    label_key=("event_remaining_time", ()),
    test=True,
)

Processing...
9453it [00:30, 313.29it/s]
Done!


In [4]:
ds_train.get_summary()

100%|██████████| 22056/22056 [00:05<00:00, 4032.01it/s]


EventGraphDataset (#graphs=22056):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     12.5 |     13.7 |
| std        |      3.6 |      4.5 |
| min        |      6   |      5   |
| quantile25 |     10   |     11   |
| median     |     12   |     13   |
| quantile75 |     14   |     16   |
| max        |     41   |     54   |
+------------+----------+----------+

In [9]:
ds_test.get_summary()

EventGraphDataset (#graphs=9453):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     12.5 |     13.7 |
| std        |      3.5 |      4.4 |
| min        |      6   |      5   |
| quantile25 |     10   |     11   |
| median     |     12   |     13   |
| quantile75 |     14   |     16   |
| max        |     38   |     50   |
+------------+----------+----------+

In [4]:
def count_parameters(model) ->int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Initialize model
model = GCN(ds_train.num_node_features, 12)
print(model)
print(f'Number of parameters: {count_parameters(model)}')


# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model = model.to(device)
# data = ds_train.to(device)

# Initialize Optimizer
learning_rate = 0.01
decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=decay)
NUM_GRAPHS_PER_BATCH = 512
# Define loss function (CrossEntropyLoss for Classification Problems with 
# probability distributions)
loss_fn = torch.nn.MSELoss()

train_loader = DataLoader(ds_train,batch_size=NUM_GRAPHS_PER_BATCH,shuffle=True)
test_loader = DataLoader(ds_test,batch_size=NUM_GRAPHS_PER_BATCH,shuffle=True)

def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n MAE: \n {mean_absolute_error(y_pred, y_true)}")
    print(f"MSE: {mean_squared_error(y_true, y_pred)}")
    print(f"R^2: {r2_score(y_true, y_pred)}")

GCN(
  (conv1): GCNConv(23, 12)
  (conv2): GCNConv(12, 12)
  (out): Linear(in_features=12, out_features=1, bias=True)
)
Number of parameters: 457
Device: cpu


In [6]:
def train_one_epoch(epoch_index:int, model, train_loader, optimizer, loss_fn, tb_writer):
    # Enumerate over the data
    running_loss = 0.0
    last_loss = 0
    for i, batch in enumerate(tqdm(train_loader)):
        # Use GPU
        batch.to(device)
        # Every data instance is an input + label pair
        inputs, adjacency_matrix, labels = batch.x.float(), batch.edge_index, batch.y.float()
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        outputs = model(inputs, adjacency_matrix) 
        # Compute loss and gradients
        loss = loss_fn(torch.squeeze(outputs), labels)
        loss.backward()
        # Adjust learnable weights
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    
    return last_loss

def run_training(
    num_epochs: int,
    model,
    train_loader,
    validation_loader,
    optimizer,
    loss_fn,
    timestamp,
    training_results_dir: str,
    seed:int = None,
):
    if seed:
        torch.manual_seed(seed)
    best_vloss = 1_000_000.
    writer = SummaryWriter(f'{training_results_dir}/{model.get_class_name()}_{timestamp[:2]}h{timestamp[2:4]}m')

    for epoch in range(num_epochs):
        print('EPOCH {}:'.format(epoch + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(
            epoch, model, train_loader, optimizer, loss_fn, writer
        )

        # We don't need gradients on to do reporting
        model.train(False)

        running_vloss = 0.0
        for i, vdata in enumerate(validation_loader):
            vdata.to(device)
            vinputs, vadjacency_matrix, vlabels = vdata.x.float(), vdata.edge_index, vdata.y.float()
            voutputs = model(vinputs,vadjacency_matrix)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                        { 'Training' : avg_loss, 'Validation' : avg_vloss },
                        epoch + 1)
        writer.flush()

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = f'{training_results_dir}/{model.get_class_name()}_{timestamp}_{epoch}.pt'
            torch.save(model.state_dict(), model_path)

In [7]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
EPOCHS = 20

run_training(EPOCHS, model, train_loader, test_loader, optimizer, loss_fn, timestamp)


EPOCH 1:


100%|██████████| 44/44 [00:06<00:00,  7.30it/s]
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6322])) that is different to the input size (torch.Size([6322, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6321])) that is different to the input size (torch.Size([6321, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.

LOSS train 0 valid 2795024154624.0
EPOCH 2:


100%|██████████| 44/44 [00:05<00:00,  7.67it/s]
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6427])) that is different to the input size (torch.Size([6427, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6367])) that is different to the input size (torch.Size([6367, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.

LOSS train 0 valid 2794286743552.0
EPOCH 3:


...

100%|██████████| 44/44 [00:05<00:00,  8.06it/s]
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6376])) that is different to the input size (torch.Size([6376, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6261])) that is different to the input size (torch.Size([6261, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.

LOSS train 0 valid 1787593097216.0
EPOCH 20:


100%|██████████| 44/44 [00:07<00:00,  5.76it/s]
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6428])) that is different to the input size (torch.Size([6428, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6220])) that is different to the input size (torch.Size([6220, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.

LOSS train 0 valid 1827074605056.0


/home/tim/Development/OCELFeatureExtractionExperiments/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6553])) that is different to the input size (torch.Size([6553, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [5]:
# Predicting with the trained model
model.load_state_dict(torch.load("models/GCN1_state_dict.pt"))


<All keys matched successfully>

In [7]:
with torch.no_grad():
    fg0 = ds_test.get(0)

        outputs =  
[model(fg.x.float(), fg.edge_index) for fg in ds_test]

tensor([3.7088e+06, 3.7088e+06, 3.6916e+06, 3.6916e+06, 3.6039e+06, 2.3990e+06,
        2.3990e+06, 2.3990e+06, 5.1466e+05, 5.1464e+05, 1.0244e+05, 1.0243e+05,
        8.3185e+04, 1.1961e+03, 1.4000e-02, 1.1000e-02, 0.0000e+00])